##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Perceptrons multicamadas para reconhecimento de dígitos com APIs Core

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/mlp_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a> </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/mlp_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/mlp_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/core/mlp_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

Este notebook usa as [APIs de baixo nível do TensorFlow Core](https://www.tensorflow.org/guide/core) para criar um fluxo de trabalho de aprendizado de máquina completo para classificação de dígitos manuscritos com [perceptrons multicamadas](https://developers.google.com/machine-learning/crash-course/introduction-to-neural-networks/anatomy) e o [dataset MNIST](http://yann.lecun.com/exdb/mnist). Veja a [Visão geral das APIs Core](https://www.tensorflow.org/guide/core) para saber mais sobre o TensorFlow Core e seus casos de uso pretendidos.

## Visão geral do perceptron multicamadas (MLP)

O Perceptron Multicamadas (Multilayer Perceptron - MLP) é um tipo de rede neural feedforward usada para abordar problemas de [classificação multiclasse](https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/video-lecture) . Antes de construir um MLP, é de fundamental importância entender os conceitos de perceptrons, camadas e funções de ativação.

Perceptrons multicamadas são compostos de unidades funcionais chamadas de perceptrons. A equação de um perceptron é a seguinte:

$$Z = \vec{w}⋅\mathrm{X} + b$$

onde

- $Z$: saída do perceptron
- $\mathrm{X}$: matriz de características
- $\vec{w}$: vetor de peso
- $b$: bias

Quando esses perceptrons são empilhados, eles formam estruturas chamadas de camadas densas, que podem ser conectadas para construir uma rede neural. A equação de uma camada densa é semelhante à de um perceptron, mas usa uma matriz de peso e um vetor de bias:

$$Z = \mathrm{W}⋅\mathrm{X} + \vec{b}$$

onde

- $Z$: saída da camada densa
- $\mathrm{X}$: matriz de características
- $\mathrm{W}$: matriz de pesos
- $\vec{b}$: vetor de bias

Numa MLP, múltiplas camadas densas são conectadas de forma tal que as saídas de uma camada sejam totalmente conectadas às entradas da camada seguinte. Adicionar funções de ativação não lineares às saídas de camadas densas pode ajudar o classificador MLP a aprender limites de decisão complexos e fazer boas generalizações para dados não vistos.

## Configuração

Importe TensorFlow, [pandas](https://pandas.pydata.org) , [Matplotlib](https://matplotlib.org) e [seaborn](https://seaborn.pydata.org) para começar.

In [ ]:
# Use seaborn for countplot.
!pip install -q seaborn

In [ ]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os
# Preset Matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
print(tf.__version__)
# Set random seed for reproducible results 
tf.random.set_seed(22)

## Carregando os dados

Este tutorial usa o [dataset MNIST](http://yann.lecun.com/exdb/mnist) e demonstra como construir um modelo MLP que pode classificar dígitos manuscritos. O dataset está disponível em [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/mnist).

Divida o conjunto de dados MNIST em conjuntos de treinamento, validação e teste. O conjunto de validação pode ser usado para avaliar a generalização do modelo durante o treinamento para que o dataset de teste possa servir como um estimador final imparcial para o desempenho do modelo.


In [ ]:
train_data, val_data, test_data = tfds.load("mnist", 
                                            split=['train[10000:]', 'train[0:10000]', 'test'],
                                            batch_size=128, as_supervised=True)

O dataset MNIST consiste em dígitos manuscritos e seus true labels (rótulos verdadeiros) correspondentes. Veja alguns exemplos abaixo.

In [ ]:
x_viz, y_viz = tfds.load("mnist", split=['train[:1500]'], batch_size=-1, as_supervised=True)[0]
x_viz = tf.squeeze(x_viz, axis=3)

for i in range(9):
    plt.subplot(3,3,1+i)
    plt.axis('off')
    plt.imshow(x_viz[i], cmap='gray')
    plt.title(f"True Label: {y_viz[i]}")
    plt.subplots_adjust(hspace=.5)


Veja também a distribuição de dígitos nos dados de treinamento para verificar se cada classe está bem representada no dataset.


In [ ]:
sns.countplot(x=y_viz.numpy());
plt.xlabel('Digits')
plt.title("MNIST Digit Distribution");

## Pré-processamento dos dados

Primeiro, remodele as matrizes de características para que sejam bidimensionais achatando as imagens. Em seguida, redimensione os dados para que os valores de pixel de [0,255] caibam no intervalo de [0,1]. Essa etapa garante que os pixels de entrada tenham distribuições semelhantes e ajuda na convergência do treinamento.

In [ ]:
def preprocess(x, y):
  # Reshaping the data
  x = tf.reshape(x, shape=[-1, 784])
  # Rescaling the data
  x = x/255
  return x, y

train_data, val_data = train_data.map(preprocess), val_data.map(preprocess)

## Construção do MLP

Comece visualizando as funções de ativação [ReLU](https://developers.google.com/machine-learning/glossary#ReLU) e [Softmax](https://developers.google.com/machine-learning/glossary#softmax) . Ambas as funções estão disponíveis em `tf.nn.relu` e `tf.nn.softmax` respectivamente. O ReLU é uma função de ativação não linear que produz na saída a entrada se for positiva e 0 caso contrário:

$$\text{ReLU}(X) = max(0, X)$$

In [ ]:
x = tf.linspace(-2, 2, 201)
x = tf.cast(x, tf.float32)
plt.plot(x, tf.nn.relu(x));
plt.xlabel('x')
plt.ylabel('ReLU(x)')
plt.title('ReLU activation function');

A função de ativação softmax é uma função exponencial normalizada que converte $m$ números reais numa distribuição de probabilidade com $m$ resultados/classes. Isto é útil para prever probabilidades de classe a partir da saída de uma rede neural:

$$\text{Softmax}(X) = \frac{e^{X}}{\sum_{i=1}^{m}e^{X_i}}$$

In [ ]:
x = tf.linspace(-4, 4, 201)
x = tf.cast(x, tf.float32)
plt.plot(x, tf.nn.softmax(x, axis=0));
plt.xlabel('x')
plt.ylabel('Softmax(x)')
plt.title('Softmax activation function');

### A camada densa

Crie uma classe para a camada densa. Por definição, as saídas de uma camada estão totalmente conectadas às entradas da camada seguinte numa MLP. Portanto, a dimensão de entrada para uma camada densa pode ser inferida com base na dimensão de saída de sua camada anterior e não precisa ser especificada antecipadamente durante a inicialização. Os pesos também devem ser inicializados corretamente para evitar que as saídas de ativação se tornem muito grandes ou pequenas. Um dos métodos de inicialização de pesos mais populares é o esquema Xavier, onde cada elemento da matriz de pesos é amostrado da seguinte maneira:

$$W_{ij} \sim \text{Uniform}(-\frac{\sqrt{6}}{\sqrt{n + m}},\frac{\sqrt{6}}{\sqrt{n + m}})$$

O vetor de bias pode ser inicializado com zeros.

In [ ]:
def xavier_init(shape):
  # Computes the xavier initialization values for a weight matrix
  in_dim, out_dim = shape
  xavier_lim = tf.sqrt(6.)/tf.sqrt(tf.cast(in_dim + out_dim, tf.float32))
  weight_vals = tf.random.uniform(shape=(in_dim, out_dim), 
                                  minval=-xavier_lim, maxval=xavier_lim, seed=22)
  return weight_vals

O método de inicialização Xavier também pode ser implementado com `tf.keras.initializers.GlorotUniform` .

In [ ]:
class DenseLayer(tf.Module):

  def __init__(self, out_dim, weight_init=xavier_init, activation=tf.identity):
    # Initialize the dimensions and activation functions
    self.out_dim = out_dim
    self.weight_init = weight_init
    self.activation = activation
    self.built = False

  def __call__(self, x):
    if not self.built:
      # Infer the input dimension based on first call
      self.in_dim = x.shape[1]
      # Initialize the weights and biases
      self.w = tf.Variable(self.weight_init(shape=(self.in_dim, self.out_dim)))
      self.b = tf.Variable(tf.zeros(shape=(self.out_dim,)))
      self.built = True
    # Compute the forward pass
    z = tf.add(tf.matmul(x, self.w), self.b)
    return self.activation(z)

Em seguida, construa uma classe para o modelo MLP que execute as camadas sequencialmente. Lembre-se de que as variáveis ​​do modelo só estão disponíveis após a primeira sequência de chamadas de camada densa devido à inferência de dimensão.

In [ ]:
class MLP(tf.Module):

  def __init__(self, layers):
    self.layers = layers
   
  @tf.function
  def __call__(self, x, preds=False): 
    # Execute the model's layers sequentially
    for layer in self.layers:
      x = layer(x)
    return x

Inicialize um modelo MLP com a seguinte arquitetura:

- Passo para a frente: ReLU (784 x 700) x ReLU (700 x 500) x Softmax (500 x 10)

A função de ativação softmax não precisa ser aplicada pelo MLP. É computada separadamente nas funções de perda e previsão.

In [ ]:
hidden_layer_1_size = 700
hidden_layer_2_size = 500
output_size = 10

mlp_model = MLP([
    DenseLayer(out_dim=hidden_layer_1_size, activation=tf.nn.relu),
    DenseLayer(out_dim=hidden_layer_2_size, activation=tf.nn.relu),
    DenseLayer(out_dim=output_size)])

### Definição da função de perda

A função de perda de entropia cruzada é uma ótima escolha para problemas de classificação multiclasse, pois mede a verossimilhança logarítmica negativa dos dados de acordo com as previsões de probabilidade do modelo. Quanto maior a probabilidade atribuída à classe verdadeira, menor a perda. A equação para a perda de entropia cruzada é a seguinte:

$$L = -\frac{1}{n}\sum_{i=1}^{n}\sum_{i=j}^{n} {y_j}^{[i]}⋅\log(\hat{{y_j}}^{[i]})$$

onde

- $\underset{n\times m}{\hat{y}}$: uma matriz de distribuições de classes previstas
- $\underset{n\times m}{y}$: uma matriz codificada (com hot encoding) de classes verdadeiras

A função `tf.nn.sparse_softmax_cross_entropy_with_logits` pode ser usada para calcular a perda de entropia cruzada. Esta função não requer que a última camada do modelo aplique a função de ativação softmax nem requer que os rótulos de classe sejam codificados com hot encoding

In [ ]:
def cross_entropy_loss(y_pred, y):
  # Compute cross entropy loss with a sparse operation
  sparse_ce = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_pred)
  return tf.reduce_mean(sparse_ce)

Escreva uma função básica de exatidão que calcule a proporção de classificações corretas durante o treinamento. Para gerar previsões de classe a partir das saídas softmax, retorne o índice que corresponde à maior probabilidade de classe. 

In [ ]:
def accuracy(y_pred, y):
  # Compute accuracy after extracting class predictions
  class_preds = tf.argmax(tf.nn.softmax(y_pred), axis=1)
  is_equal = tf.equal(y, class_preds)
  return tf.reduce_mean(tf.cast(is_equal, tf.float32))

### Treinamento do modelo

O uso de um otimizador pode resultar numa convergência significativamente mais rápida em comparação com o método do gradiente descendente padrão. O otimizador Adam é implementado abaixo. Consulte o guia [Otimizadores](https://www.tensorflow.org/guide/core/optimizers_core) para saber mais sobre como criar otimizadores personalizados com o TensorFlow Core.

In [ ]:
class Adam:

    def __init__(self, learning_rate=1e-3, beta_1=0.9, beta_2=0.999, ep=1e-7):
      # Initialize optimizer parameters and variable slots
      self.beta_1 = beta_1
      self.beta_2 = beta_2
      self.learning_rate = learning_rate
      self.ep = ep
      self.t = 1.
      self.v_dvar, self.s_dvar = [], []
      self.built = False
      
    def apply_gradients(self, grads, vars):
      # Initialize variables on the first call
      if not self.built:
        for var in vars:
          v = tf.Variable(tf.zeros(shape=var.shape))
          s = tf.Variable(tf.zeros(shape=var.shape))
          self.v_dvar.append(v)
          self.s_dvar.append(s)
        self.built = True
      # Update the model variables given their gradients
      for i, (d_var, var) in enumerate(zip(grads, vars)):
        self.v_dvar[i].assign(self.beta_1*self.v_dvar[i] + (1-self.beta_1)*d_var)
        self.s_dvar[i].assign(self.beta_2*self.s_dvar[i] + (1-self.beta_2)*tf.square(d_var))
        v_dvar_bc = self.v_dvar[i]/(1-(self.beta_1**self.t))
        s_dvar_bc = self.s_dvar[i]/(1-(self.beta_2**self.t))
        var.assign_sub(self.learning_rate*(v_dvar_bc/(tf.sqrt(s_dvar_bc) + self.ep)))
      self.t += 1.
      return 

Agora, escreva um loop de treinamento personalizado que atualize os parâmetros MLP com o método do gradiente descendente em minilotes. O uso de minilotes para treinamento garante uma eficiência de memória e convergência mais rápida.

In [ ]:
def train_step(x_batch, y_batch, loss, acc, model, optimizer):
  # Update the model state given a batch of data
  with tf.GradientTape() as tape:
    y_pred = model(x_batch)
    batch_loss = loss(y_pred, y_batch)
  batch_acc = acc(y_pred, y_batch)
  grads = tape.gradient(batch_loss, model.variables)
  optimizer.apply_gradients(grads, model.variables)
  return batch_loss, batch_acc

def val_step(x_batch, y_batch, loss, acc, model):
  # Evaluate the model on given a batch of validation data
  y_pred = model(x_batch)
  batch_loss = loss(y_pred, y_batch)
  batch_acc = acc(y_pred, y_batch)
  return batch_loss, batch_acc

In [ ]:
def train_model(mlp, train_data, val_data, loss, acc, optimizer, epochs):
  # Initialize data structures
  train_losses, train_accs = [], []
  val_losses, val_accs = [], []

  # Format training loop and begin training
  for epoch in range(epochs):
    batch_losses_train, batch_accs_train = [], []
    batch_losses_val, batch_accs_val = [], []

    # Iterate over the training data
    for x_batch, y_batch in train_data:
      # Compute gradients and update the model's parameters
      batch_loss, batch_acc = train_step(x_batch, y_batch, loss, acc, mlp, optimizer)
      # Keep track of batch-level training performance
      batch_losses_train.append(batch_loss)
      batch_accs_train.append(batch_acc)

    # Iterate over the validation data
    for x_batch, y_batch in val_data:
      batch_loss, batch_acc = val_step(x_batch, y_batch, loss, acc, mlp)
      batch_losses_val.append(batch_loss)
      batch_accs_val.append(batch_acc)

    # Keep track of epoch-level model performance
    train_loss, train_acc = tf.reduce_mean(batch_losses_train), tf.reduce_mean(batch_accs_train)
    val_loss, val_acc = tf.reduce_mean(batch_losses_val), tf.reduce_mean(batch_accs_val)
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    val_losses.append(val_loss)
    val_accs.append(val_acc)
    print(f"Epoch: {epoch}")
    print(f"Training loss: {train_loss:.3f}, Training accuracy: {train_acc:.3f}")
    print(f"Validation loss: {val_loss:.3f}, Validation accuracy: {val_acc:.3f}")
  return train_losses, train_accs, val_losses, val_accs

Treine o modelo MLP por 10 épocas com tamanho de lote de 128. Aceleradores de hardware como GPUs ou TPUs também podem ajudar a acelerar o tempo de treinamento. 

In [ ]:
train_losses, train_accs, val_losses, val_accs = train_model(mlp_model, train_data, val_data, 
                                                             loss=cross_entropy_loss, acc=accuracy,
                                                             optimizer=Adam(), epochs=10)

### Avaliação de desempenho

Comece escrevendo uma função de plotagem para visualizar a perda e a precisão do modelo durante o treinamento. 

In [ ]:
def plot_metrics(train_metric, val_metric, metric_type):
  # Visualize metrics vs training Epochs
  plt.figure()
  plt.plot(range(len(train_metric)), train_metric, label = f"Training {metric_type}")
  plt.plot(range(len(val_metric)), val_metric, label = f"Validation {metric_type}")
  plt.xlabel("Epochs")
  plt.ylabel(metric_type)
  plt.legend()
  plt.title(f"{metric_type} vs Training epochs");

In [ ]:
plot_metrics(train_losses, val_losses, "cross entropy loss")

In [ ]:
plot_metrics(train_accs, val_accs, "accuracy")

## Salvando e carregando o modelo

Comece criando um módulo de exportação que receba dados brutos e execute as seguintes operações:

- Pré-processamento de dados
- Previsão de probabilidade
- Previsão de classe

In [ ]:
class ExportModule(tf.Module):
  def __init__(self, model, preprocess, class_pred):
    # Initialize pre and postprocessing functions
    self.model = model
    self.preprocess = preprocess
    self.class_pred = class_pred

  @tf.function(input_signature=[tf.TensorSpec(shape=[None, None, None, None], dtype=tf.uint8)]) 
  def __call__(self, x):
    # Run the ExportModule for new data points
    x = self.preprocess(x)
    y = self.model(x)
    y = self.class_pred(y)
    return y 

In [ ]:
def preprocess_test(x):
  # The export module takes in unprocessed and unlabeled data
  x = tf.reshape(x, shape=[-1, 784])
  x = x/255
  return x

def class_pred_test(y):
  # Generate class predictions from MLP output
  return tf.argmax(tf.nn.softmax(y), axis=1)

Este módulo de exportação agora pode ser salvo com a função `tf.saved_model.save`. 

In [ ]:
mlp_model_export = ExportModule(model=mlp_model,
                                preprocess=preprocess_test,
                                class_pred=class_pred_test)

In [ ]:
models = tempfile.mkdtemp()
save_path = os.path.join(models, 'mlp_model_export')
tf.saved_model.save(mlp_model_export, save_path)

Carregue o modelo salvo com `tf.saved_model.load` e examine seu desempenho nos dados de teste não vistos.

In [ ]:
mlp_loaded = tf.saved_model.load(save_path)

In [ ]:
def accuracy_score(y_pred, y):
  # Generic accuracy function
  is_equal = tf.equal(y_pred, y)
  return tf.reduce_mean(tf.cast(is_equal, tf.float32))

x_test, y_test = tfds.load("mnist", split=['test'], batch_size=-1, as_supervised=True)[0]
test_classes = mlp_loaded(x_test)
test_acc = accuracy_score(test_classes, y_test)
print(f"Test Accuracy: {test_acc:.3f}")

O modelo faz um ótimo trabalho de classificação de dígitos manuscritos no dataset de treinamento e também generaliza bem para dados não vistos. Agora, examine a precisão de classe do modelo para garantir um bom desempenho para cada dígito. 

In [ ]:
print("Accuracy breakdown by digit:")
print("---------------------------")
label_accs = {}
for label in range(10):
  label_ind = (y_test == label)
  # extract predictions for specific true label
  pred_label = test_classes[label_ind]
  labels = y_test[label_ind]
  # compute class-wise accuracy
  label_accs[accuracy_score(pred_label, labels).numpy()] = label
for key in sorted(label_accs):
  print(f"Digit {label_accs[key]}: {key:.3f}")

Parece que o modelo luta com alguns dígitos um pouco mais do que outros, o que é bastante comum em muitos problemas de classificação multiclasse. Como exercício final, desenhe uma matriz de confusão das previsões do modelo e seus true labels correspondentes para reunir mais insights em nível de classe. Sklearn e seaborn têm funções para gerar e visualizar matrizes de confusão. 

In [ ]:
import sklearn.metrics as sk_metrics

def show_confusion_matrix(test_labels, test_classes):
  # Compute confusion matrix and normalize
  plt.figure(figsize=(10,10))
  confusion = sk_metrics.confusion_matrix(test_labels.numpy(), 
                                          test_classes.numpy())
  confusion_normalized = confusion / confusion.sum(axis=1, keepdims=True)
  axis_labels = range(10)
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.4f', square=True)
  plt.title("Confusion matrix")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

show_confusion_matrix(y_test, test_classes)

Os insights em nível de classe podem ajudar a identificar motivos para erros de classificação e melhorar o desempenho do modelo em futuros ciclos de treinamento.

## Conclusão

Este notebook apresentou algumas técnicas para lidar com um problema de classificação multiclasse usando uma [MLP](https://developers.google.com/machine-learning/crash-course/multi-class-neural-networks/softmax). Aqui estão mais algumas dicas que podem ser úteis:

- As [APIs do TensorFlow Core](https://www.tensorflow.org/guide/core) podem ser usadas para criar workflows de aprendizado de máquina com altos níveis de configurabilidade
- Os esquemas de inicialização podem ajudar a evitar que os parâmetros do modelo desapareçam ou explodam durante o treinamento.
- O overfitting é outro problema comum para redes neurais, embora não tenha sido um problema neste tutorial. Consulte o tutorial [Overfit e underfit](overfit_and_underfit.ipynb) para mais informações sobre esse tema.

Para obter mais exemplos de uso das APIs Core do TensorFlow, confira o [guia](https://www.tensorflow.org/guide/core). Se você quiser saber mais sobre como carregar e preparar dados, veja os tutoriais sobre [carregamento de dados de imagem](https://www.tensorflow.org/tutorials/load_data/images) ou [carregamento de dados CSV](https://www.tensorflow.org/tutorials/load_data/csv).